<a href="https://colab.research.google.com/github/FatemehPasban/Loanword_Identification/blob/main/Root_Finder_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install hazm
!pip install transformers

     |████████████████████████████████| 316 kB 5.2 MB/s 
     |████████████████████████████████| 1.4 MB 48.0 MB/s 
     |████████████████████████████████| 233 kB 49.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394483 sha256=1ca792655713a900aedcc2824a8957dcb58fba61e37ef0cedf6839e112dc1168
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154389 sha256=80d664894038902e6ce750b936a322536d522d7262a06ae9aa43612f6d2f41b7
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 55.8 MB/s 
     |

In [ ]:
import pandas as pd
import numpy as np
from os.path import join
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from hazm import *
from transformers import BertConfig, BertTokenizer,BertModel,AutoTokenizer

import torch
import torch.utils.data as data_utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import precision_score


## config

In [ ]:
#model_name = 'HooshvareLab/bert-fa-zwnj-base'
#model_name = 'HooshvareLab/distilbert-fa-zwnj-base'
#model_name = 'HooshvareLab/albert-fa-zwnj-base-v2'
model_name = 'HooshvareLab/bert-fa-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name)

root_path ='/content/drive/MyDrive/uni2_project/Root Finder Model/'
data_path=join(root_path,'words.xlsx')
#data_path=join(root_path,'word2.xlsx')
config = BertConfig.from_pretrained(
    model_name)

Downloading:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

# The data

In [ ]:
raw_data = pd.read_excel(data_path)
raw_data

,word,syn1,syn2,syn3,syn4,syn5,syn6,syn7,syn8,syn9,syn10,syn11,syn12,syn13,syn14,syn15,syn16,syn17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,آب جاری,رواناب,آب روان,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,آباء و اجداد,پدران,نیاکان,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,آتیه,آینده,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,آثار,نشانه‌ها,نوشته‌ها,کارها,ردپا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,آحاد,یکایک,تک تک,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,یأس,ناامیدی,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4201,یبوست,خشکی,دیرگواری,سخت گواری,بدگواری,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4202,یحیی,می‌زید,زنده است,زندگی می‌کند,نامی پسرانه,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4203,یعنی,یا اینکه,روشنتر آنکه,برابر با اینکه,یا می‌گوییم,به زبان دیگر,به سخن دیگر,به گفته دیگر,یا می‌توان گفت,یا همانکه,یا آنکه (از ستاک ساختگی سه گانی برگرفته از مع...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def parse_data(row):
  data_list =[]
  row_val = row.values
  Syns = row_val[0]
  data_list.append({'word' : list( Syns[0].strip()) ,'label' : 1})
  for syn_word in Syns[1:]:
    if type(syn_word) is str:
      syn_word = syn_word.strip()
      data_list.append({'word' : list(syn_word),'label' : 0})
  return data_list

a = parse_data(raw_data[1:2])
a

[{'label': 1,
  'word': ['آ', 'ب', 'ا', 'ء', ' ', 'و', ' ', 'ا', 'ج', 'د', 'ا', 'د']},
 {'label': 0, 'word': ['پ', 'د', 'ر', 'ا', 'ن']},
 {'label': 0, 'word': ['ن', 'ی', 'ا', 'ک', 'ا', 'ن']}]

In [ ]:

parsed = [parse_data(raw_data[i:i+1]) for i in range(len(raw_data))] # nested list => [ [{},{}] , [{}] , [{},{},{}] , ...]
parsed = [i for item in parsed for i in item] # merging nested list => [ {} , {} , {} , ... ]
df_data = pd.DataFrame(parsed)
df_data

,word,label
0,"[آ, ب, , ج, ا, ر, ی]",1
1,"[ر, و, ا, ن, ا, ب]",0
2,"[آ, ب, , ر, و, ا, ن]",0
3,"[آ, ب, ا, ء, , و, , ا, ج, د, ا, د]",1
4,"[پ, د, ر, ا, ن]",0
...,...,...
15756,"[ی, ا, , م, ی, ‌, ت, و, ا, ن, , گ, ف, ت]",0
15757,"[ی, ا, , ه, م, ا, ن, ک, ه]",0
15758,"[ی, ا, , آ, ن, ک, ه, , (, ا, ز, , س, ت, ا, ...",0
15759,"[ی, ق, ی, ن]",1


In [ ]:
df_data.groupby('label').count()

,word
label,
0,11556
1,4205


### Equalize data in the direction of the label

In [ ]:
df_dataa = shuffle(df_data)
df_data0 = df_dataa[df_dataa.label == 0]
df_data1 = df_dataa[df_dataa.label == 1]

df_data_new = shuffle( pd.concat ([df_data0[:len(df_data1)] , df_data1] , axis=0) )
#df_data_new = pd.concat ([ df_data0[:len(df_data1)] , df_data1 ]) 
df_data_new

,word,label
6463,"[خ, ن, ث, ی]",1
4758,"[ت, م, س, ا, ح]",1
6501,"[خ, ی, ا, ل, , پ, ر, د, ا, ز, ا, ن, ه]",1
2491,"[ب, د, , ذ, ا, ت]",1
14262,"[ه, م, س, ا, ز, ی]",0
...,...,...
10299,"[ف, ا, ر, غ, ‌, ا, ل, ت, ح, ص, ی, ل]",1
8473,"[ص, ن, ع, ت]",1
1766,"[پ, و, ش, ا, ک]",0
11765,"[و, ر, ج, ا, و, ن, د]",0


In [ ]:
df_data_new.groupby('label').count()

,word
label,
0,4205
1,4205


## Encoding the Dataset with the Tokenizer

In [ ]:
encode_words = [tokenizer.encode(text)
                for text in df_data_new['word']]
df_data_new['encoded_words'] = encode_words
df_data_new

#Some time it has
#ValueError: Input [] is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

,word,label,encoded_words
6463,"[خ, ن, ث, ی]",1,"[2, 1359, 1377, 1356, 1442, 4]"
4758,"[ت, م, س, ا, ح]",1,"[2, 1355, 1376, 1364, 1352, 1358, 4]"
6501,"[خ, ی, ا, ل, , پ, ر, د, ا, ز, ا, ن, ه]",1,"[2, 1359, 1442, 1352, 1375, 1, 1407, 1362, 136..."
2491,"[ب, د, , ذ, ا, ت]",1,"[2, 1353, 1360, 1, 1361, 1352, 1355, 4]"
14262,"[ه, م, س, ا, ز, ی]",0,"[2, 1378, 1376, 1364, 1352, 1363, 1442, 4]"
...,...,...,...
10299,"[ف, ا, ر, غ, ‌, ا, ل, ت, ح, ص, ی, ل]",1,"[2, 1373, 1352, 1362, 1371, 1, 1352, 1375, 135..."
8473,"[ص, ن, ع, ت]",1,"[2, 1366, 1377, 1370, 1355, 4]"
1766,"[پ, و, ش, ا, ک]",0,"[2, 1407, 1379, 1365, 1352, 1426, 4]"
11765,"[و, ر, ج, ا, و, ن, د]",0,"[2, 1379, 1362, 1357, 1352, 1379, 1377, 1360, 4]"


In [ ]:
#join(root_path,"dataframee.xlsx")

In [ ]:
#writer = pd.ExcelWriter(join(root_path,"dataframee2.xlsx"))
#df_data_new.to_excel(writer,encoding='utf8')
#writer.save()

In [ ]:
data_sequence_lengths = [len(text)
                         for text in df_data_new['encoded_words']]
max_length = max(data_sequence_lengths)
MAX_LEN = max_length + 8
MAX_LEN

45

##  Split data

In [ ]:
df_train,df_test = train_test_split(df_data_new)
first_sentense = df_train.iloc[0]['word']
first_sentense

['خ', 'ر', 'د', 'س', 'ا', 'ل', 'ی']

In [ ]:
writer_train = pd.ExcelWriter(join(root_path,"df_big_train.xlsx"))
writer_test = pd.ExcelWriter(join(root_path,"df_big_test.xlsx"))
df_train.to_excel(writer_train,encoding='utf8')
df_test.to_excel(writer_test,encoding='utf8')
writer_train.save()
writer_test.save()

In [ ]:
df_train.iloc[0]['encoded_words']

[2, 1359, 1362, 1360, 1364, 1352, 1375, 1442, 4]

In [ ]:
def encode_dataset(tokenizer, text_sequences, max_length):
    token_ids = np.zeros(shape=(len(text_sequences), max_length),
                         dtype=np.int32)
    for i, text_sequence in enumerate(text_sequences):
        encoded = tokenizer.encode(text_sequence)
        token_ids[i, 0:len(encoded)] = encoded
    attention_masks = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_masks": attention_masks}


encoded_test = encode_dataset(tokenizer, df_test["word"], MAX_LEN)
encoded_train = encode_dataset(tokenizer, df_train["word"], MAX_LEN)
encoded_train

{'attention_masks': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32),
 'input_ids': array([[   2, 1359, 1362, ...,    0,    0,    0],
        [   2, 1352, 1377, ...,    0,    0,    0],
        [   2, 1370, 1361, ...,    0,    0,    0],
        ...,
        [   2, 1373, 1366, ...,    0,    0,    0],
        [   2, 1352, 1377, ...,    0,    0,    0],
        [   2, 1362, 1365, ...,    0,    0,    0]], dtype=int32)}

In [ ]:
encoded_train["attention_masks"][0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int32)

In [ ]:
test_labels = df_test['label'].values
train_labels = df_train['label'].values
len(train_labels),train_labels

(6307, array([0, 1, 1, ..., 1, 0, 1]))

## Create torch DataLoader

In [ ]:
Batch_size = 32
train_labels_tensor = torch.utils.data.DataLoader(train_labels , batch_size=Batch_size , shuffle=False)
test_labels_tensor = torch.utils.data.DataLoader(test_labels , batch_size=Batch_size , shuffle=False)
for l in enumerate(test_labels_tensor):
  print(l[1])
  ll = l[1]

tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 0, 1, 0])
tensor([0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 0])
tensor([0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1])
tensor([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 1, 1])
tensor([0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 0])
tensor([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 0])
tensor([0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 1])
tensor([0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 1,

In [ ]:
#train_data_tensor_input_ids = torch.tensor(encoded_train['input_ids'])
train_data_tensor_input_ids = torch.utils.data.DataLoader(encoded_train['input_ids'], batch_size=Batch_size , shuffle=False)
#train_data_tensor_input_ids2 = next(iter(train_data_tensor_input_ids))

#train_data_tensor_attention_masks = torch.tensor(encoded_train['attention_masks'])
train_data_tensor_attention_masks = torch.utils.data.DataLoader(encoded_train['attention_masks'], batch_size=Batch_size , shuffle=False)
#train_data_tensor_attention_masks2 = next(iter(train_data_tensor_attention_masks))

#test_data_tensor_input_ids = torch.tensor(encoded_test['input_ids'])
test_data_tensor_input_ids = torch.utils.data.DataLoader(encoded_test['input_ids'], batch_size=Batch_size , shuffle=False)
#test_data_tensor_input_ids2 = next(iter(test_data_tensor_input_ids))

#test_data_tensor_attention_masks = torch.tensor(encoded_test['attention_masks'])
test_data_tensor_attention_masks = torch.utils.data.DataLoader(encoded_test['attention_masks'],batch_size=Batch_size , shuffle=False)
#test_data_tensor_attention_masks2 = next(iter(test_data_tensor_attention_masks))


#test_data_loader = torch.utils.data.DataLoader(encoded_test, 10, True)
#test_data_loader2 = next(iter(test_data_tensor_attention_masks))

train_data_tensor_input_ids

In [ ]:
num_batches = len(df_train)// Batch_size
num_batches

197

just to know, how to iterate through batches :

In [ ]:
#for input_ids_batch ,attention_masks_batch in zip(enumerate(test_data_tensor_input_ids),enumerate(test_data_tensor_attention_masks)):
#  print(input_ids_batch,attention_masks_batch[1])
#  print('============')
#  input_ids_batches = input_ids_batch[1]
#input_ids_batches

# Classification

## Model

In [ ]:
class RootClassifierModel(nn.Module):
    

    def __init__(self, config):
        super(RootClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(model_name,return_dict=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.Root_classifier = nn.Linear(config.hidden_size, 2 )
    
    def forward(self, input_ids, attention_mask):
    #def forward(self, input):
        sequence_output , pooled_output = self.bert(
            #input
            input_ids=input_ids, 
            attention_mask=attention_mask
            )
        pooled_output = self.dropout(pooled_output)
        Root_logits = self.Root_classifier(pooled_output)

        return Root_logits

In [ ]:
Root_model = RootClassifierModel(config=config)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(params=Root_model.parameters(),lr=0.0008)
#optimizer = optim.Adam(params=Root_model.parameters())
optimizer = optim.Adam(params=Root_model.parameters(), lr=3e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

## Training

In [ ]:
#1
#2
#3
#4
num_epoch = 1
for epoch in range(num_epoch):
  b=0
  Root_model.train()

  score_list=[]
  for input_ids_batch ,attention_masks_batch ,labels_batch in zip(enumerate(train_data_tensor_input_ids),enumerate(train_data_tensor_attention_masks),enumerate(train_labels_tensor)):

    optimizer.zero_grad()
    output = Root_model(input_ids_batch[1],attention_masks_batch[1])
    #print(output.shape,labels_batch[1].shape)

    loss = criterion(output,labels_batch[1])
    loss.backward()
    optimizer.step()

    _, preds = torch.max(output, dim=1)
    nu=0
    for j in range(len(preds)):
      if preds[j]==labels_batch[1][j]:
        nu+=1
    score = (nu/len(preds)) * 100
    score_list.append(score)

    b+=1
    print(f'epoch {epoch+1}/{num_epoch} , batch {b}/{num_batches+1} loss = {loss.item()}')
    print(f'batch precision score : {score} % ')
    #print(f'precision {precision_score(labels_batch[1],preds)}')
    print('----------')


epoch 1/1 , batch 1/198 loss = 0.21703565120697021
batch precision score : 87.5 % 
----------
epoch 1/1 , batch 2/198 loss = 0.04010931774973869
batch precision score : 100.0 % 
----------
epoch 1/1 , batch 3/198 loss = 0.08418770879507065
batch precision score : 96.875 % 
----------
epoch 1/1 , batch 4/198 loss = 0.08770041167736053
batch precision score : 96.875 % 
----------
epoch 1/1 , batch 5/198 loss = 0.08896567672491074
batch precision score : 93.75 % 
----------
epoch 1/1 , batch 6/198 loss = 0.10250198841094971
batch precision score : 93.75 % 
----------
epoch 1/1 , batch 7/198 loss = 0.028582777827978134
batch precision score : 100.0 % 
----------
epoch 1/1 , batch 8/198 loss = 0.02791781537234783
batch precision score : 100.0 % 
----------
epoch 1/1 , batch 9/198 loss = 0.2178158164024353
batch precision score : 90.625 % 
----------
epoch 1/1 , batch 10/198 loss = 0.23809072375297546
batch precision score : 93.75 % 
----------
epoch 1/1 , batch 11/198 loss = 0.1243808791041

## evaluation

In [ ]:
i=0
num_batches_test = len(df_test)// Batch_size  + 1
score_list =[]
for input_ids_batch ,attention_masks_batch,labels in zip(enumerate(test_data_tensor_input_ids),enumerate(test_data_tensor_attention_masks),enumerate(test_labels_tensor)):
  output2 = Root_model(input_ids_batch[1],attention_masks_batch[1])
  _, preds = torch.max(output2, dim=1)
  i+=1
  nu = 0
  for j in range(len(preds)):
    if preds[j]==labels[1][j]:
      nu+=1
  score = (nu/len(preds)) * 100
  score_list.append(score)

  print(i,nu,'/',len(preds))
  print(f'batch precision score : {score} % ')
  print('-------')
  #if i == 5:
  # break
total_score = sum(score_list)/len(score_list)
print(f'total score is : {total_score}')

1 30 / 32
batch precision score : 93.75 % 
-------
2 29 / 32
batch precision score : 90.625 % 
-------
3 27 / 32
batch precision score : 84.375 % 
-------
4 31 / 32
batch precision score : 96.875 % 
-------
5 30 / 32
batch precision score : 93.75 % 
-------
6 31 / 32
batch precision score : 96.875 % 
-------
7 30 / 32
batch precision score : 93.75 % 
-------
8 31 / 32
batch precision score : 96.875 % 
-------
9 30 / 32
batch precision score : 93.75 % 
-------
10 28 / 32
batch precision score : 87.5 % 
-------
11 27 / 32
batch precision score : 84.375 % 
-------
12 29 / 32
batch precision score : 90.625 % 
-------
13 30 / 32
batch precision score : 93.75 % 
-------
14 32 / 32
batch precision score : 100.0 % 
-------
15 29 / 32
batch precision score : 90.625 % 
-------
16 31 / 32
batch precision score : 96.875 % 
-------
17 30 / 32
batch precision score : 93.75 % 
-------
18 26 / 32
batch precision score : 81.25 % 
-------
19 28 / 32
batch precision score : 87.5 % 
-------
20 29 / 32
bat

In [ ]:
#1 => total score is : 89.57509881422925
#2 => total score is : 91.63990447957839
#3 => total score is : 91.73460144927536

### Save model

In [ ]:
model_save_name = 'RootClassifierModel_bigdataa.pt'
#path = F"/content/drive/MyDrive/uni2_project/{model_save_name}" 
path = root_path + model_save_name
torch.save(Root_model, path)